 # Quantization Aware Training <a class="anchor" id="head-0"></a>
 
In this section, we will explore the typical Quantization-Aware Training(QAT) workflow with TLT. QAT workflow is almost the same as non-QAT workflow except for two major differences:
1. set `enable_qat` to `True` in training and retraining spec files to enable the QAT for training/retraining
2. when doing export in INT8 mode, the calibration cache is extracted directly from the QAT .tlt model, so no need to specify any TensorRT INT8 calibration related arguments for `export`

 ## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env GPU_INDEX=0
%env KEY=tlt

# Please define this local project directory that needs to be mapped to the TLT docker session.
%env LOCAL_PROJECT_DIR=/home/astr1x/Projects/FasterRCNN-TLT-Triton
os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "faster_rcnn"
)
%env USER_EXPERIMENT_DIR=/workspace/tlt-experiments/faster_rcnn
%env DATA_DOWNLOAD_DIR=/workspace/tlt-experiments/data
# The sample spec files are present in the same path as the downloaded samples.
# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tlt-samples/faster_rcnn
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tlt-experiments/faster_rcnn/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TLT docker instance, so that the data and the results are mapped from outside to inside of the docker instance.

In [ ]:
# Mapping up the local directories to the TLT docker.
import json
import os
mounts_file = os.path.expanduser("~/.tlt_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tlt-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    # set gpu index for tlt-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ],
    "DockerOptions":{
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tlt_mounts.json

 ## 1. Prepare dataset <a class="anchor" id="head-2"></a>

In [ ]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/coco_val_skimmed.zip -d $LOCAL_DATA_DIR

In [ ]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/images")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/labels")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "training/images")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

In [ ]:
print("TFrecords conversion spec file for training")
!cat $LOCAL_SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt

In [ ]:
# Creating a new directory for the output tfrecords dump.
!mkdir -p $LOCAL_EXPERIMENT_DIR/tfrecords
#KITTI trainval
!tlt faster_rcnn dataset_convert --gpu_index $GPU_INDEX -d $SPECS_DIR/frcnn_tfrecords_kitti_trainval.txt \
                     -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_trainval/kitti_trainval

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/kitti_trainval

 ## 2. Download pretrained model <a class="anchor" id="head-2"></a>

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_reg_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli --no-check-certificate
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list "nvidia/tlt_pretrained_object_detection*"

In [ ]:
# Download model from NGC.
!ngc registry model download-version nvidia/tlt_pretrained_object_detection:resnet18

In [ ]:
# Copy weights to experiment directory.
!cp tlt_pretrained_object_detection_vresnet18/resnet_18.hdf5 $LOCAL_EXPERIMENT_DIR
!rm -rf tlt_pretrained_object_detection_vresnet18
!ls -rlt $LOCAL_EXPERIMENT_DIR

 ## 3. Training <a class="anchor" id="head-10.1"></a>

In [ ]:
# set enable_qat to True in training spec file to enable QAT training
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18.txt

In [ ]:
# run QAT training
!tlt faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ## 4. Evaluation <a class="anchor" id="head-10.2"></a>

In [ ]:
!tlt faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18.txt

 ## 5. Pruning <a class="anchor" id="head-10.3"></a>

In [ ]:
!tlt faster_rcnn prune --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18.epoch12.tlt \
           -o $USER_EXPERIMENT_DIR/model_1_pruned.tlt  \
           -eq union  \
           -pth 0.2 \
           -k $KEY

 ## 6. Retraining <a class="anchor" id="head-10.4"></a>

In [ ]:
# set enable_qat to True in retraining spec file to enable QAT
!sed -i 's/enable_qat: False/enable_qat: True/' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!cat $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
!tlt faster_rcnn train --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 7. Evaluation of the retrained model <a class="anchor" id="head-10.5"></a>

In [ ]:
# disable the tensorrt evaluation config in spec file
!TRT_LINES=$(grep -n 'trt_evaluation' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/^/#/g\n' $(seq $TRT_LINES $((TRT_LINES+4))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# do evaluation with .tlt model
!tlt faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

 ## 8. Inference of the retrained model <a class="anchor" id="head-10.6"></a>

In [ ]:
# disable the tensorrt inference config in spec file
!TRT_LINES=$(grep -n 'trt_inference' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/^/#/g\n' $(seq $TRT_LINES $((TRT_LINES+4))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# do inference with .tlt model
!tlt faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Visualizing the sample images
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ## 9. Deployment of the QAT model <a class="anchor" id="head-10.7"></a>

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# No need for calibration dataset for QAT model INT8 export
!if [ -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt ]; then rm -f $LOCAL_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt; fi
!if [ -f $LOCAL_EXPERIMENT_DIR/cal.bin ]; then rm -f $LOCAL_EXPERIMENT_DIR/cal.bin; fi
!tlt faster_rcnn export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain.epoch12.tlt  \
                        -o $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt \
                        -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt \
                        -k $KEY \
                        --data_type int8 \
                        --cal_cache_file $USER_EXPERIMENT_DIR/cal.bin

In [ ]:
# Convert to TensorRT engine(INT8).
# Make sure your GPU type supports the INT8 data type before running this cell.
!tlt tlt-converter -k $KEY  \
               -d 3,384,1248 \
               -o NMS \
               -c $USER_EXPERIMENT_DIR/cal.bin \
               -e $USER_EXPERIMENT_DIR/trt.int8.engine \
               -b 8 \
               -m 4 \
               -t int8 \
               -i nchw \
               $USER_EXPERIMENT_DIR/frcnn_kitti_resnet18_retrain_int8_qat.etlt

In [ ]:
print('Exported model and converted TensorRT engine:')
print('------------')
!ls -lht $LOCAL_EXPERIMENT_DIR

In [ ]:
# Do inference with TensorRT on the generated TensorRT engine
# Please go to $LOCAL_EXPERIMENT_DIR/inference_results_imgs_retrain to see the visualizations.
!TRT_LINES=$(grep -n 'trt_inference' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+4))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tlt faster_rcnn inference --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt

In [ ]:
# Visualizing the sample images from TensorRT inference.
OUTPUT_PATH = 'inference_results_imgs_retrain' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# Doing evaluation with the generated TensorRT engine
# compare the mAP below with that of `evaluate` with retrained tlt model
!TRT_LINES=$(grep -n 'trt_evaluation' $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt | cut -d: -f1) && printf '%ds/#//g\n' $(seq $TRT_LINES $((TRT_LINES+4))) | sed -i -f - $LOCAL_SPECS_DIR/default_spec_resnet18_retrain_spec.txt
!tlt faster_rcnn evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/default_spec_resnet18_retrain_spec.txt